In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

#model evaluation
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, average_precision_score, roc_curve, auc

import pickle


# load data

In [34]:
# #load full data
# data = pd.read_csv('data/compas-scores-train.csv')  

# #split train-test 70-30
# train, test = train_test_split(data, test_size=0.30, random_state=123)

# #save splits
# train.to_csv('data/compas-scores-train-70.csv', index=False)
# test.to_csv('data/compas-scores-test-30.csv', index=False)

train = pd.read_csv('german-train.csv')
test = pd.read_csv('german-test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'german-train.csv'

In [35]:
#load split data
data_train = train
data_test = test

display(data_train.head())

print('data, train:', data_train.shape)
print('data, test:', data_test.shape)

#split into X, y
X_train = data_train.loc[:, data_train.columns != 'credit-risk']
y_train = data_train['credit-risk']

X_test = data_test.loc[:, data_test.columns != 'credit-risk']
y_test = data_test['credit-risk']


X=X_train
y=y_train
print('----- TRAIN -----')
print('X, shape:', X.shape)
print('y, shape:', y.shape)
print('#class1: ', sum(y), f', prop = {sum(y)/len(y)}')
print('#class0:', sum(y==0), f', prop = {sum(y==0)/len(y)}')

X=X_test
y=y_test
print('----- TEST -----')
print('X, shape:', X.shape)
print('y, shape:', y.shape)
print('#class1: ', sum(y), f', prop = {sum(y)/len(y)}')
print('#class0:', sum(y==0), f', prop = {sum(y==0)/len(y)}')

,status,duration,credit-history,purpose,amount,savings,employment-duration,installment-rate,personal-status-sex,other-debtors,...,property,age,other-installment-plans,housing,number-credits,job,people-liable,telephone,foreign-worker,credit-risk
0,1,15,3,4,1213,4,5,4,5,1,...,3,47,2,3,1,3,1,2,1,1
1,2,24,3,4,2439,2,2,4,2,1,...,4,35,3,3,1,3,1,2,1,0
2,1,60,3,1,10366,2,5,2,5,1,...,3,42,3,3,1,4,1,2,1,1
3,4,6,1,7,1047,2,3,2,2,1,...,3,50,3,3,1,2,1,1,1,1
4,2,18,3,4,1882,2,3,4,2,1,...,2,25,1,2,2,3,1,1,1,0


data, train: (800, 21)
data, test: (200, 21)
----- TRAIN -----
X, shape: (800, 20)
y, shape: (800,)
#class1:  560 , prop = 0.7
#class0: 240 , prop = 0.3
----- TEST -----
X, shape: (200, 20)
y, shape: (200,)
#class1:  140 , prop = 0.7
#class0: 60 , prop = 0.3


# model 1: logistic regression

In [36]:
#model 1: logistic regression

#fit model
model_logistic = LogisticRegression(penalty='none').fit(X_train, y_train)

#save model
model_filename = 'model_logistic.pkl'
pickle.dump(model_logistic, open(model_filename, 'wb'))

# #load model
# model_logistic = pickle.load(open(model_filename, 'rb'))

/Users/javinpombra/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
def evaluate_model(y_true, y_pred, y_prob_class1):
    #overall
    print('acccuracy:', accuracy_score(y_true, y_pred))
    fpr, tpr, thresholds = roc_curve(y_true, y_prob_class1) #y_prob
    print('AUC:', auc(fpr, tpr))
    
    #class 1
    print('\n***class 1***')
    pos_label = 1
    print('recall:', recall_score(y_true, y_pred, pos_label=pos_label))
    print('precision:', precision_score(y_true, y_pred, pos_label=pos_label))
    print('F1 score:', f1_score(y_true, y_pred, pos_label=pos_label))
    print('AP:', average_precision_score(y_true, y_pred, pos_label=pos_label))
    
    #class 0
    print('\n***class 0***')
    pos_label = 0
    print('recall:', recall_score(y_true, y_pred, pos_label=pos_label))
    print('precision:', precision_score(y_true, y_pred, pos_label=pos_label))
    print('F1 score:', f1_score(y_true, y_pred, pos_label=pos_label))
    print('AP:', average_precision_score(y_true, y_pred, pos_label=pos_label))

In [38]:
#evaluate model

###training set
X=X_train
y=y_train
print('----- TRAIN -----')
evaluate_model(y_true=y, 
               y_pred=model_logistic.predict(X), 
               y_prob_class1=model_logistic.predict_proba(X)[:, 1])

###test set
X=X_test
y=y_test
print('\n----- TEST -----')
evaluate_model(y_true=y, 
               y_pred=model_logistic.predict(X), 
               y_prob_class1=model_logistic.predict_proba(X)[:, 1])

----- TRAIN -----
acccuracy: 0.73375
AUC: 0.7378199404761905

***class 1***
recall: 0.9125
precision: 0.7570370370370371
F1 score: 0.8275303643724697
AP: 0.7520462962962964

***class 0***
recall: 0.31666666666666665
precision: 0.608
F1 score: 0.4164383561643835
AP: 0.2610246913580247

----- TEST -----
acccuracy: 0.735
AUC: 0.7192857142857144

***class 1***
recall: 0.8928571428571429
precision: 0.7668711656441718
F1 score: 0.8250825082508251
AP: 0.7597063978965819

***class 0***
recall: 0.36666666666666664
precision: 0.5945945945945946
F1 score: 0.4536082474226804
AP: 0.2576482617586912


# Model 2: gradient boosted tree

In [39]:
#model 2: gradient boosted tree

#train model
model_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#use_label_encoder=False, remove warning (no impact on model performance)
#eval_metric='logloss', remove warning (this is the default, but need to specify)
model_gb.fit(X_train, y_train)

#save model
model_filename = 'model_gb.pkl'
pickle.dump(model_gb, open(model_filename, 'wb'))

#load model
# model_gb = pickle.load(open(model_filename, 'rb'))


In [40]:
#evaluate model

###training set
X=X_train
y=y_train
print('----- TRAIN -----')
evaluate_model(y_true=y, 
               y_pred=model_gb.predict(X), 
               y_prob_class1=model_gb.predict_proba(X)[:, 1])

###test set
X=X_test
y=y_test
print('\n----- TEST -----')
evaluate_model(y_true=y, 
               y_pred=model_gb.predict(X), 
               y_prob_class1=model_gb.predict_proba(X)[:, 1])

----- TRAIN -----
acccuracy: 0.85125
AUC: 0.8872916666666668

***class 1***
recall: 0.9267857142857143
precision: 0.8693467336683417
F1 score: 0.8971477960242005
AP: 0.8569481335247667

***class 0***
recall: 0.675
precision: 0.7980295566502463
F1 score: 0.7313769751693003
AP: 0.24496231155778897

----- TEST -----
acccuracy: 0.685
AUC: 0.7110714285714285

***class 1***
recall: 0.7857142857142857
precision: 0.7692307692307693
F1 score: 0.7773851590106008
AP: 0.7543956043956045

***class 0***
recall: 0.45
precision: 0.47368421052631576
F1 score: 0.46153846153846156
AP: 0.2619230769230769


# Model 3: random forest

In [41]:
#model 3: random forest

#train model
model_rf = RandomForestClassifier(n_estimators=50, random_state=12345)
model_rf.fit(X_train, y_train)

#save model
model_filename = 'model_rf.pkl'
pickle.dump(model_rf, open(model_filename, 'wb'))

# #load model
# model_rf = pickle.load(open(model_filename, 'rb'))


In [42]:
#evaluate model

###training set
X=X_train
y=y_train
print('----- TRAIN -----')
evaluate_model(y_true=y, 
               y_pred=model_rf.predict(X), 
               y_prob_class1=model_rf.predict_proba(X)[:, 1])

###test set
X=X_test
y=y_test
print('\n----- TEST -----')
evaluate_model(y_true=y, 
               y_pred=model_rf.predict(X), 
               y_prob_class1=model_rf.predict_proba(X)[:, 1])

----- TRAIN -----
acccuracy: 1.0
AUC: 0.9999999999999999

***class 1***
recall: 1.0
precision: 1.0
F1 score: 1.0
AP: 1.0

***class 0***
recall: 1.0
precision: 1.0
F1 score: 1.0
AP: 0.3

----- TEST -----
acccuracy: 0.75
AUC: 0.7748809523809524

***class 1***
recall: 0.8428571428571429
precision: 0.8082191780821918
F1 score: 0.8251748251748252
AP: 0.7912133072407045

***class 0***
recall: 0.5333333333333333
precision: 0.5925925925925926
F1 score: 0.5614035087719299
AP: 0.24949771689497718


# Model 4: neural network

In [43]:
#model 4: neural network

In [44]:
#create COMPASDataset class
class GermanDataset(Dataset):
    
    def __init__(self, X, y):
        self.X = X.to_numpy()
        self.y = y.to_numpy()
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]
    
#create datasets
train_ds = GermanDataset(X_train, y_train)
test_ds = GermanDataset(X_test, y_test)

#create dataloaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64)

In [45]:
#create model class
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, seed=12345):
        super().__init__()
        
        torch.manual_seed(seed)
        
        #variables
        self.input_size = input_size
        self.hidden_size = hidden_size
        #layers architecture
        self.linear_layer1 = nn.Linear(self.input_size, self.hidden_size)
        self.linear_layer2 = nn.Linear(self.hidden_size, self.hidden_size*2)
        self.linear_layer3 = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.linear_layer4 = nn.Linear(self.hidden_size, 1)
        
    def forward(self, inputs):
        out = self.linear_layer1(inputs)
        out = nn.functional.relu(out)
        out = self.linear_layer2(out)
        out = nn.functional.relu(out)
        out = self.linear_layer3(out)
        out = nn.functional.relu(out)
        out = self.linear_layer4(out)
        out = torch.sigmoid(out)
        return out
    
    def predict_proba(self, X):
        X = torch.tensor(X).type(torch.FloatTensor)
        class1_probs = self.forward(X).detach().numpy()
        class0_probs = 1-class1_probs
        return np.hstack((class0_probs, class1_probs))
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


In [46]:
#train model

#model
model = FFNN(input_size=X_train.shape[1], hidden_size=50)

n_epochs = 20
seed = 12345
torch.manual_seed(seed)
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()

#function to calculate accuracy
def compute_n_correct(y_true, pred_prob): 
    y_pred = (pred_prob.squeeze()>0.5)*1
    return sum(y_pred == y_true)

train_losses = []
train_accs = []
#val_losses = []
#val_accs = []


#loop through epochs and batches
for epoch in range(n_epochs):
    running_loss = 0
    running_n_correct = 0
    
    for batch_input, batch_output in train_dl:
        batch_input = batch_input.type(torch.FloatTensor)
        batch_output = batch_output.type(torch.FloatTensor)
        #forward pass: compute model output and loss
        preds = model(batch_input)
        loss = loss_fn(preds.squeeze(), batch_output)
        #backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        #store metrics
        running_loss += loss.cpu().item()
        n_correct = compute_n_correct(y_true=batch_output, pred_prob=preds)
        running_n_correct += n_correct
    
    train_losses.append(running_loss / len(train_ds))
    train_accs.append(running_n_correct / len(train_ds))
    print('-'*20)
    print(f'Epoch {epoch+1}/{n_epochs} Train Loss: {running_loss / len(train_ds)}')
    print(f'Epoch {epoch+1}/{n_epochs} Train Accuracy: {running_n_correct / len(train_ds)}')


--------------------
Epoch 1/20 Train Loss: 0.0820277202129364
Epoch 1/20 Train Accuracy: 0.5950000286102295
--------------------
Epoch 2/20 Train Loss: 0.023356277644634247
Epoch 2/20 Train Accuracy: 0.6012499928474426
--------------------
Epoch 3/20 Train Loss: 0.011985910087823869
Epoch 3/20 Train Accuracy: 0.625
--------------------
Epoch 4/20 Train Loss: 0.011995783001184463
Epoch 4/20 Train Accuracy: 0.6087499856948853
--------------------
Epoch 5/20 Train Loss: 0.011473262161016464
Epoch 5/20 Train Accuracy: 0.6612499952316284
--------------------
Epoch 6/20 Train Loss: 0.011735741868615151
Epoch 6/20 Train Accuracy: 0.6549999713897705
--------------------
Epoch 7/20 Train Loss: 0.015516375154256822
Epoch 7/20 Train Accuracy: 0.6387500166893005
--------------------
Epoch 8/20 Train Loss: 0.016177776753902435
Epoch 8/20 Train Accuracy: 0.6225000023841858
--------------------
Epoch 9/20 Train Loss: 0.014225016236305238
Epoch 9/20 Train Accuracy: 0.5987499952316284
----------------

In [47]:
#save model
model_filename = 'model_nn.pkl'
torch.save(model, model_filename)


In [48]:
#evaluate model

###training set
X=train_ds.X
y=train_ds.y
print('----- TRAIN -----')
evaluate_model(y_true=y, 
               y_pred=model.predict(X), 
               y_prob_class1=model.predict_proba(X)[:, 1])

###test set
X=test_ds.X
y=test_ds.y
print('\n----- TEST -----')
evaluate_model(y_true=y, 
               y_pred=model.predict(X), 
               y_prob_class1=model.predict_proba(X)[:, 1])

----- TRAIN -----
acccuracy: 0.7
AUC: 0.5784747023809523

***class 1***
recall: 1.0
precision: 0.7
F1 score: 0.8235294117647058
AP: 0.7

***class 0***
recall: 0.0
precision: 0.0
F1 score: 0.0
AP: 0.3

----- TEST -----
acccuracy: 0.7
AUC: 0.5947619047619048

***class 1***
recall: 1.0
precision: 0.7
F1 score: 0.8235294117647058
AP: 0.7

***class 0***
recall: 0.0
precision: 0.0
F1 score: 0.0
AP: 0.3


/Users/javinpombra/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/javinpombra/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model 5: logistic regression as NN

In [49]:
#create model class
class LogisticRegressionNN(nn.Module):
    def __init__(self, input_size, seed=12345):
        super().__init__()
        
        torch.manual_seed(seed)
        
        #variables
        self.input_size = input_size
        #layers
        self.linear_layer = nn.Linear(self.input_size, 1)
        
    def forward(self, inputs):
        out = self.linear_layer(inputs)
        out = torch.sigmoid(out)
        return out
    
    def predict_proba(self, X):
        X = torch.tensor(X).type(torch.FloatTensor)
        class1_probs = self.forward(X).detach().numpy()
        class0_probs = 1-class1_probs
        return np.hstack((class0_probs, class1_probs))
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)



In [50]:
#logistic regression parameters, from logistic regression
print(model_logistic.coef_)
print(model_logistic.intercept_)


[[-3.91054484e-01 -2.35488197e-02 -2.06097371e-01  5.47986878e-02
  -9.10443864e-05  8.66636869e-02  2.07817391e-01 -2.64174471e-01
   2.30962910e-01  1.49323424e-02 -9.82170969e-02  1.69672842e-01
   1.48541158e-02  1.45873137e-01  1.70009429e-01  1.31618389e-02
   1.99936028e-02 -8.76016820e-03  1.20377910e-01  3.70709776e-02]]
[0.00820714]


In [51]:
#change NN model weights to logistic regression coefficient

#instantiate model
model_nn_logistic = LogisticRegressionNN(input_size=X_train.shape[1])

#change NN model weights
lr_coefs = torch.tensor(model_logistic.coef_, requires_grad=True).type(torch.FloatTensor)
lr_intercept = torch.tensor(model_logistic.intercept_, requires_grad=True).type(torch.FloatTensor)

model_nn_logistic.linear_layer.weight = nn.Parameter(lr_coefs)
model_nn_logistic.linear_layer.bias = nn.Parameter(lr_intercept)

print(model_nn_logistic.linear_layer.weight)
print(model_nn_logistic.linear_layer.bias)

Parameter containing:
tensor([[-3.9105e-01, -2.3549e-02, -2.0610e-01,  5.4799e-02, -9.1044e-05,
          8.6664e-02,  2.0782e-01, -2.6417e-01,  2.3096e-01,  1.4932e-02,
         -9.8217e-02,  1.6967e-01,  1.4854e-02,  1.4587e-01,  1.7001e-01,
          1.3162e-02,  1.9994e-02, -8.7602e-03,  1.2038e-01,  3.7071e-02]],
       requires_grad=True)
Parameter containing:
tensor([0.0082], requires_grad=True)


In [52]:
#save model
model_filename = 'model_nn_logistic.pkl'
torch.save(model_nn_logistic, model_filename)

#load model
# model_nn_logistic = torch.load(model_filename)


In [53]:
#evaluate model -- same results as logistic regression model (model #1)

###training set
X=train_ds.X
y=train_ds.y
print('----- TRAIN -----')
evaluate_model(y_true=y, 
               y_pred=model.predict(X), 
               y_prob_class1=model.predict_proba(X)[:, 1])

###test set
X=test_ds.X
y=test_ds.y
print('\n----- TEST -----')
evaluate_model(y_true=y, 
               y_pred=model.predict(X), 
               y_prob_class1=model.predict_proba(X)[:, 1])

----- TRAIN -----
acccuracy: 0.7
AUC: 0.5784747023809523

***class 1***
recall: 1.0
precision: 0.7
F1 score: 0.8235294117647058
AP: 0.7

***class 0***
recall: 0.0
precision: 0.0
F1 score: 0.0
AP: 0.3

----- TEST -----
acccuracy: 0.7
AUC: 0.5947619047619048

***class 1***
recall: 1.0
precision: 0.7
F1 score: 0.8235294117647058
AP: 0.7

***class 0***
recall: 0.0
precision: 0.0
F1 score: 0.0
AP: 0.3


/Users/javinpombra/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/javinpombra/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
